# Note
This notebook is to train the encoder as a classifier with the idea of validate the encoder architecture first and then use this to train the VAE.

# Parameters to experiment

In [1]:
# training on guanaco
# ATENTION: if it is going to run on guanaco:
# 1) comment the %matplotlib magic in next block and any magic (something like %code)
# 2) Change to True the trainingOnGuanaco vairbale
# 3) set epoch with an appropiate number
# 4) add comment to experiemnts
# 5) Add this file as python file 
# 6) Change launchJobOnGuanaco file to run this file but with python format
trainingOnGuanaco = False

# train without notebook
trainWithJustPython = False

# number_experiment (this is just a name)
# priors:
# 1
number_experiment = 99
number_experiment = str(number_experiment)

# seed to generate same datasets
seed = 0

# training
epochs = 3

# max elements by class
# max_elements_per_class = 15000 # this is the definitive in the work
max_elements_per_class = 15000

# train with previous model
trainWithPreviousModel = True

# include delta errors
includeDeltaErrors = True

# band
#passband = [5]
passband = [0, 1, 2, 3, 4, 5]


# include ohter feautures
includeOtherFeatures = True

# num of features to add
# ṕvar by channel
otherFeaturesDim = 12

In [2]:
# to analuze execution times
# useGpu = False
useGpu = True

In [3]:
# cuda device
cuda_device = 0
cuda_device = "cuda:" + str(cuda_device)

# classes to analyze
# 42,  90,  16,  67,  62, 993,  92,  52,  88,  65, 991, 992,  15,
#        95,   6,  53, 994,  64

# periodic
# only_these_labels = [16, 92, 53]

# periodic + variable
only_these_labels = [16, 92, 53, 88, 65, 6]
# 53 has 24 light curves

# only_these_labels = [16, 92]
# only_these_labels = [16, 92]
# only_these_labels = [42,  90,  16,  67,  62, 993,  92,  52,  88,  65, 991, 992,  15,
#         95,   6,  53, 994,  64]

# VAE parameters
latentDim = 100
hiddenDim = 100
inputDim = 72

batch_training_size = 128

# early stopping 
threshold_early_stop = 1500

In [4]:
# training params
learning_rate = 1e-4

In [5]:
# add general comment about experiment 
# comment = "encoder as clasifier with periodic + variable (with class balancing) + 1 conv layer more"
comment = "exp " + number_experiment + " + encoder as clasifier with periodic + variable + class balancing + 1 conv layer more + " + str(len(passband)) + " channels + seed " + str(seed) + " + " + ("include delta errors" if includeDeltaErrors else "without delta errors") + " + max by class " + str(max_elements_per_class) + " + " + ("" if includeOtherFeatures else "not") + " other features"

print(comment)

exp 99 + encoder as clasifier with periodic + variable + class balancing + 1 conv layer more + 6 channels + seed 0 + include delta errors + max by class 15000 +  other features


# Import libraries

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch.utils import data

# from tqdm import tqdm_notebook

if not trainingOnGuanaco:
    
    %matplotlib notebook
    %load_ext autoreload
    %autoreload 2
else:
    print("not load magics")
    
# import functions to load dataset
import sys
sys.path.append("./codesToDatasets")
from plasticc_dataset_torch import get_plasticc_datasets
from plasticc_plotting import plot_light_curve

import math

from torch import nn

# local imports
# %load_ext autoreload
# %autoreload 2
sys.path.append('../models')
# from classifier import EncoderClassifier, 
from classifierPrototype import EncoderClassifier

sys.path.append("./aux/")
from auxFunctions import *

from sklearn.model_selection import train_test_split

In [56]:
# libraries for RF
import os
os.environ["MKL_NUM_THREADS"]="1"
# print(os.environ["MKL_NUM_THREADS"])
import sys
from os.path import join, exists
import pandas as pd
from joblib import Parallel, delayed, dump
import pickle
from itertools import zip_longest 
import turbofats
import torch
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Load the path to save model while training

In [7]:
import os

# create experiment's folder
tmpGuanaco = "/home/lbravo/thesis/thesis/work/thesis/"
tmpLocal = "/home/leo/Desktop/thesis/work/thesis/"

# expPath = "experiments/" + number_experiment + "/seed" + str(seed) + "/maxClass" + str(int(max_elements_per_class/1000)) + "k"
# this si to use the 15k model but using other max of classes as dataset
# expPath = "experiments/" + number_experiment + "/seed" + str(seed) + "/maxClass" + str(15) + "k"
# 18 is the final model
expPath = "experiments/" + str(number_experiment) + "/seed" + str(seed) + "/maxClass" + str(15) + "k"

folder_path = (tmpGuanaco + expPath) if trainingOnGuanaco else (tmpLocal + expPath)


# check if folder exists
if not(os.path.isdir(folder_path)):
        
    # create folder
    try:
        os.makedirs(folder_path)
        
    except OSError as error:
        print ("Creation of the directory %s failed" % folder_path)
        print(error)
    else:
        print ("Successfully created the directory %s " % folder_path)
else:
    print("folder already exists")

    
    
# # define paht to save model while training
# pathToSaveModel = (tmpGuanaco + expPath + "/model") if trainingOnGuanaco else (tmpLocal + expPath + "/model")

Successfully created the directory /home/leo/Desktop/thesis/work/thesis/experiments/99/seed0/maxClass15k 


# Load data

In [8]:
# define path to dataset
pathToFile = "/home/shared/astro/PLAsTiCC/" if trainingOnGuanaco else "/home/leo/Downloads/plasticData/"

## Loading dataset with pytorch tool

In [9]:
# torch_dataset_lazy = get_plasticc_datasets(pathToFile)

# Light curves are tensors are now [bands, [mjd, flux, err, mask],
# lc_data, lc_label, lc_plasticc_id                              
torch_dataset_lazy = get_plasticc_datasets(pathToFile, only_these_labels=only_these_labels, max_elements_per_class = max_elements_per_class)

You have selected lazy loading. Light curves will be loaded ondemand from the harddrive
Found 2 csv files at given path
Loading /home/leo/Downloads/plasticData/plasticc_train_lightcurves.csv
Loading /home/leo/Downloads/plasticData/plasticc_test_set_batch1.csv


# Spliting data (train/test)

In [10]:
# splitting the data

# get light curves ids, targets
ids, targets, lightCurvesIds = getLightCurvesIds(torch_dataset_lazy)

# test array shapes
# assert len(targets) == torch_dataset_lazy.__len__()
# print(ids, len(ids), targets, len(targets))
# get light curves targets
print("# light curves ids: " + str(len(ids)))

# split training
trainIdx, tmpIdx = train_test_split(
    ids,
    test_size = 0.2,
    shuffle = True,
    stratify = targets,
    random_state = seed
)

# float to int
tmpIdx = tmpIdx.astype(int)

# split val, test
valIdx, testIdx = train_test_split(
    tmpIdx,
#     targets,
    test_size = 0.5,
    shuffle = True,
    stratify = targets[tmpIdx],
    random_state = seed
)

# float to int
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)
testIdx = testIdx.astype(int)

# light curves ids: 3276


In [11]:
# saving ids
saveLightCurvesIdsBeforeBalancing(trainIdx, valIdx, testIdx, folder_path, lightCurvesIds, targets)

light curves ids saved on a file


In [12]:
# # Spliting the data

# # print(torch_dataset_lazy.__len__())

totalSize = torch_dataset_lazy.__len__()

# totalSize = totalSize
# # print(totalSize)

# selecting train splitting
# train_size = int(0.8 * totalSize)
train_size = trainIdx.shape[0]
#print(train_size)

# # getting test splitting
# validation_size = math.floor((totalSize - train_size)/3)
validation_size = valIdx.shape[0]
# #print(validation_size)

# # getting test splitting
# test_size = totalSize - train_size - validation_size
test_size = testIdx.shape[0]
# #print(test_size)

print("train size:", train_size)
print("validation size: ", validation_size)
print("test size:", test_size)
totTmp = train_size+ validation_size + test_size
print("sum: ", totTmp)
assert torch_dataset_lazy.__len__() == totTmp, "dataset partition should be the same"

train size: 2620
validation size:  328
test size: 328
sum:  3276


## Create a dataloader

In [13]:
# # # Create data loader (minibatches)

# # training loader
# trainLoader = torch.utils.data.DataLoader(
#     torch_dataset_lazy, 
# #     batch_size = batch_training_size, 
#     # to balance classes
#     sampler=ImbalancedDatasetSampler(
#         torch_dataset_lazy, 
#         indices = trainIdx,
#         seed = seed
# #         indices = [0, 1, 2]
#     ),
# #     sampler = torch.utils.data.SubsetRandomSampler(
# #         trainIdx,
# #         generator = torch.Generator().manual_seed(seed)
# #     ),
#     # each worker retrieve data from disk, so the data will be ready to be processed by main process. The main process should get the data from disk, so if workers > 0, the workers will get the data (not the main process)
#     num_workers = 4,
    
#     # https://developer.nvidia.com/blog/how-optimize-data-transfers-cuda-cc/
#     # the dataloader loads the data in pinned memory (instead of pageable memory), avoiding one process (to transfer data from pageable memory to pinned memory, work done by CUDA driver)
#     pin_memory = True,
# )


# # validation loader
# validationLoader = torch.utils.data.DataLoader(
# #     validationDataset, 
#     torch_dataset_lazy,
# #     batch_size= batch_training_size,  
#     num_workers = 4,
#     pin_memory = True,
#     sampler = valIdx,
# #     sampler = torch.utils.data.SubsetRandomSampler(
# #         valIdx,
# #         generator = torch.Generator().manual_seed(seed)
# #     ),
# #     sampler=ImbalancedDatasetSampler(
# #         torch_dataset_lazy, 
# #         indices = valIdx,
# #         seed = seed
# # #         indices = [0, 1, 2]
# #     ),
# )

# # test loader
# testLoader = torch.utils.data.DataLoader(testDataset)
testLoader = torch.utils.data.DataLoader(
#     validationDataset, 
    torch_dataset_lazy,
#     batch_size= batch_training_size,  
    num_workers = 4,
    pin_memory = True,
    sampler = testIdx,
#     sampler = torch.utils.data.SubsetRandomSampler(
#         testIdx,
#         generator = torch.Generator().manual_seed(seed)
#     ),
)

## Defining parameters to Autoencoder

In [14]:
# check number of parameters
# latentDim = 5
# hiddenDim = 10
# inputDim = 72

latentDim = latentDim
hiddenDim = hiddenDim
inputDim = inputDim

# passband = passband

expPath_ = "experiments/18/seed" + str(seed) + "/maxClass" + str(15) + "k"
pathToSaveModel_ = (tmpGuanaco + expPath_ + "/model") if trainingOnGuanaco else (tmpLocal + expPath_ + "/model")


num_classes = len(only_these_labels)

if useGpu:
    
    print("gpu")
    
    # loading model
    model = torch.load(pathToSaveModel_ + ".txt").to(device = cuda_device)
    
    print("loading saved model")
    

else:
    print("cpu")
    model = torch.load(pathToSaveModel_ + ".txt").cpu()

gpu
loading saved model


In [15]:
print(model)

EncoderClassifier(
  (pconv1): PartialConv(
    (input_conv): Conv1d(6, 64, kernel_size=(3,), stride=(2,))
    (mask_conv): Conv1d(6, 64, kernel_size=(3,), stride=(2,), bias=False)
  )
  (pconv2): PartialConv(
    (input_conv): Conv1d(64, 32, kernel_size=(3,), stride=(2,))
    (mask_conv): Conv1d(64, 32, kernel_size=(3,), stride=(2,), bias=False)
  )
  (pconv3): PartialConv(
    (input_conv): Conv1d(32, 32, kernel_size=(3,), stride=(2,))
    (mask_conv): Conv1d(32, 32, kernel_size=(3,), stride=(2,), bias=False)
  )
  (hidden1): Linear(in_features=768, out_features=100, bias=True)
  (outputLayer): Linear(in_features=100, out_features=6, bias=True)
  (activationConv): ReLU()
  (activationLinear): Tanh()
)


# Get own model predictions

In [15]:
print("train size: " + str(train_size))

train size: 2620


# Train

In [41]:
%%timeit -n 1 -r 1
# iterate on test dataset
# for data_ in trainLoader:


if useGpu:
    
    for idx, data_ in enumerate(testLoader):

            print("using GPU")

            # GPU
            data = data_[0].to(device = cuda_device)
            # print(data.get_device())
            data = generateDeltas(data.to(device = cuda_device), passband, includeOtherFeatures).type(torch.FloatTensor).to(device = cuda_device)

            # get model output
            outputs = model.forward(data, includeDeltaErrors)

else:
    
    for idx, data_ in enumerate(testLoader):

            
            print("using CPU")
            
            #CPU
            data = data_[0]
    #         print(data.get_device())
            data = generateDeltas(data, passband, includeOtherFeatures).type(torch.FloatTensor)

            # get model output
            outputs = model.forward(data, includeDeltaErrors)

predictions ready
7.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Stop execution if it's on cluster

In [17]:
import sys

if  trainingOnGuanaco or trainWithJustPython:

    sys.exit("Exit from code, because we are in cluster or running locally. Training has finished.")

# features + RF

In [ ]:
# get data

In [16]:
from joblib import load, dump
# # load file
rf = load('../experiments/comparingModels/seed' + str(seed) + '/RF/trainedRF.joblib')

## compute features

In [59]:
%%timeit -n 1 -r 1
#plasticc_path = "/home/shared/astro/PLAsTiCC/"
plasticc_path = "/home/leo/Downloads/plasticData/"

print("RF")
def compute_fats_features(batch_names):
    """
    Receives a list of file names and a path

    Returns a dataframe with the features
    """
    # TODO: STUDY BIASED FEATURES
    feature_list = ['CAR_sigma','CAR_mean', 'Meanvariance', 'Mean',                          'PercentAmplitude', 'Skew', 'AndersonDarling', 'Std', 'Rcs', 'StetsonK',
                         'MedianAbsDev', 'Q31', 'Amplitude', 'PeriodLS_v2', 'Harmonics',
                'Autocor_length', 'SlottedA_length', 'StetsonK_AC',  'Con', 'Beyond1Std',
                'SmallKurtosis', 'MaxSlope','MedianBRP', 'PairSlopeTrend', 
                'LinearTrend', 'Eta_e', 'Period_fit_v2', 'PeriodPowerRate',
                'Psi_CS_v2', 'Psi_eta_v2', 'StructureFunction_index_21', 'Pvar', 'StructureFunction_index_31',
                'ExcessVar', 'IAR_phi']
#     features = []
#     print(len(feature_list))
    df_features = pd.DataFrame()
    
#     print(df_features.shape)
    
#     print(batch_names)
    for name in batch_names:
#         print(name)
        # Check that name is valid
        if name is None:
            continue
        # Read light curve
        lc_data = parse_light_curve_data(name)
#         print(lc_data)
        if lc_data is None:
            continue
            
        # shape: [6 (channels) x 48 (features) = 288]
        features_lc = []
        
        # each filter or channel
        for fid in range(6):
            
            # Check that lc has more than 10 points
            #if lc_data.shape[0] < 7:
            #    print("Light curve %s has less than 7 points, skipping" %(name))
            #    continue
            
            
            # Compute features
            
            # filter by channel
            df_lc_fid = lc_data.loc[lc_data.fid == fid][["mjd", "mag", "err"]]
            
            # compute feature
            feature_space = turbofats.FeatureSpace(feature_list=feature_list,
                                                      data_column_names=["mag", "mjd", "err"])
            features_fid = feature_space.calculate_features(df_lc_fid)
            
            # rename column on data
            # shape: [48]
            features_fid = features_fid.rename(lambda x: x+"_"+str(fid), axis='columns')
            
#             print(features_fid.shape)
            
            features_lc.append(features_fid)
            
#         pd.concat()

        # concat all features of all channel of light curve
        dfOneLC = pd.concat(features_lc, axis=1)
#         print(dfOneLC.shape)
        
        # concat the light curve to full dataset
        df_features = pd.concat([df_features, dfOneLC])
#         print(df_features.shape)
        
#     # concat all data
#     print(df_features.shape)
    
    
    # do inference
    predictions = rf.predict(df_features.fillna(-1000).values)
    
#     print(predictions.shape)
    
def split_list_in_chunks(iterable, chunk_size, fillvalue=None):
    """
    Receives an iterable object (i.e. list) and a chunk size

    Returns an iterable object with the same elements on of the original but arranged in chunks
    """
    args = [iter(iterable)] * chunk_size
    return zip_longest(*args, fillvalue=fillvalue)

def parse_light_curve_data(light_curve_id):
    path_to_light_curve = join(plasticc_path, "light_curves", str(int(light_curve_id))+".pt")
    if not exists(path_to_light_curve):
        if raise_error:
            raise FileNotFoundError("File not found at: %s" %(path_to_light_curve))
        return None
    with open(path_to_light_curve, "rb") as f:
        lc_torch = torch.load(f)
    data = []
    for band in range(6):
        mask = lc_torch[band, -1, :] == 1
        tmp = lc_torch[band, :3, mask].T
        data.append(np.concatenate((tmp.numpy(), np.ones(shape=(tmp.shape[0], 1))*band),axis=1))
    df_lc = pd.DataFrame(data=np.concatenate(data, axis=0), columns=['mjd', 'mag', 'err', 'fid'])
    df_lc.index = [int(light_curve_id)]*len(df_lc.index.unique())
    return df_lc


if __name__ == "__main__":


    #with open("/home/phuijse/plasticc_rf/dataset_ids_before_balancing.pkl", "rb") as f:
    with open("/home/leo/Desktop/thesis/work/thesis/experiments/" + str(number_experiment) +"/seed" + str(seed) + "/maxClass15k/dataset_ids_before_balancing.pkl", "rb") as f:
        ids = pickle.load(f)
#         print(ids["validation"].shape)
#     for subset in ["train", "validation", "test"]:
#     for subset in ["test"]:
#         print(subset)
        subset = "test"
        Parallel(n_jobs=1)(delayed(compute_fats_features)(batch_names) for batch_names in split_list_in_chunks(ids[subset], 100))
#         result = [compute_fats_features(ids[subset][:2])]
#         print("result")
#         print(result)
#         with open(join(plasticc_path, "features/features_fats_"+subset+".pkl"), "wb") as f:
#            dump(result, f, protocol=3)

RF


KeyboardInterrupt: 